In [3]:
import pandas as pd
import numpy as np

In [4]:
titles = pd.read_csv('Titles.tsv', sep='\t')
titles.head()

C:\Users\zbtin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


Checking for null values

In [18]:
titles.title.isna().value_counts()

False    22543552
True            2
Name: title, dtype: int64

In [19]:
titles.titleId.isna().value_counts()

False    22543554
Name: titleId, dtype: int64

In [20]:
titles.region.isna().value_counts()

False    22543465
True           89
Name: region, dtype: int64

Dropping null values

In [31]:
titles.dropna(inplace = True)

In [32]:
titles.shape

(22543463, 8)

In [35]:
titles.isOriginalTitle.value_counts()

0     13058629
0      9008568
1       374061
1       100070
\N        2135
Name: isOriginalTitle, dtype: int64

In [36]:
titles['isOriginalTitle'].replace('0', 0, inplace = True)
titles['isOriginalTitle'].replace('1', 1, inplace = True)
titles.isOriginalTitle.value_counts()

0     22067197
1       474131
\N        2135
Name: isOriginalTitle, dtype: int64

In [39]:
titles.region.value_counts().head(20)

FR     2789011
JP     2774882
DE     2743378
ES     2724525
IT     2711374
IN     2690382
PT     2662052
US     1060544
\N      533292
GB      277483
CA      133900
XWW     127899
BR       83975
AU       83058
GR       72032
MX       70984
FI       65493
RU       63537
HU       58550
PL       56332
Name: region, dtype: int64

Chloreopleth map can be created using this

In [41]:
titles.types.value_counts()

\N                      20893381
imdbDisplay               996495
original                  474132
alternative                70948
working                    44147
dvd                        19918
tv                         15707
video                      15478
festival                   12787
dvdimdbDisplay              232
imdbDisplaytv                70
festivalimdbDisplay          68
imdbDisplayworking           31
imdbDisplayvideo             25
alternativetv                 9
alternativedvd                6
alternativeworking            6
tvvideo                       6
videoworking                  5
tvworking                     4
festivalworking               3
alternativevideo              2
alternativefestival           1
dvdvideo                      1
dvdworking                    1
Name: types, dtype: int64

In [43]:
titles.language.value_counts().head(20)

\N     3830471
ja     2679053
fr     2644268
hi     2617721
es     2614816
de     2614694
it     2613618
pt     2613467
en      196422
ru       25468
bg       20587
tr       20066
cmn       9931
sv        7633
he        6412
qbn       4939
sr        3966
yue       3810
fa        2387
ca        2084
Name: language, dtype: int64

In [44]:
titles.attributes.value_counts()

\N                                   22350358
transliterated title                    24000
alternative spelling                    13215
literal English title                   12374
new title                               10512
                                       ...   
twentyeighth season title                   1
fourth season titlerecut version           1
thirtyfifth season title                    1
Hakka dialect title                         1
POLartvideo box title                      1
Name: attributes, Length: 188, dtype: int64

Since language, attributes and type have a predominance of null values we will not be using them to estimate the ratings later

In [46]:
titles.drop(columns=['attributes', 'language', 'types'], inplace = True)

In [47]:
titles.shape

(22543463, 5)

Cleaning of titles dataset complete

Cleaning of ratings dataset 

In [49]:
ratings = pd.read_csv('Ratings.tsv', sep='\t')
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.6,1639
1,tt0000002,6.1,198
2,tt0000003,6.5,1331
3,tt0000004,6.2,120
4,tt0000005,6.1,2114


In [50]:
ratings.shape

(1059926, 3)

In [55]:
df = ratings.set_index('tconst').join(titles.set_index('titleId'), how='left')

In [56]:
df.shape

(2983481, 6)

In [60]:
df

,averageRating,numVotes,ordering,title,region,isOriginalTitle
tt0000001,5.6,1639,1.0,Карменсіта,UA,0
tt0000001,5.6,1639,2.0,Carmencita,DE,0
tt0000001,5.6,1639,3.0,Carmencita - spanyol tánc,HU,0
tt0000001,5.6,1639,4.0,Καρμενσίτα,GR,0
tt0000001,5.6,1639,5.0,Карменсита,RU,0
...,...,...,...,...,...,...
tt9916766,6.9,14,4.0,Folge #10.15,DE,0
tt9916766,6.9,14,5.0,Episódio #10.15,PT,0
tt9916766,6.9,14,6.0,एपिसोड #10.15,IN,0
tt9916766,6.9,14,7.0,Épisode #10.15,FR,0


We notice that the many movies have multiple entries, just diffenent title and region

Let's see which region has the most analyzable movies

In [61]:
df.region.value_counts()

US    390413
\N    326489
FR    145416
JP    133590
DE    128584
       ...  
AI         1
NU         1
SB         1
VC         1
PW         1
Name: region, Length: 230, dtype: int64

Given the news IMDb contains it is not unexpected that the US region will dominate

To remove multiple releases, we will just concentrate on the US movies

In [66]:
US_region = df['region']=='US'
df = df[US_region]
df

,averageRating,numVotes,ordering,title,region,isOriginalTitle
tt0000001,5.6,1639,6.0,Carmencita,US,0
tt0000002,6.1,198,7.0,The Clown and His Dogs,US,0
tt0000005,6.1,2114,1.0,Blacksmithing Scene,US,0
tt0000005,6.1,2114,5.0,Blacksmith Scene #1,US,0
tt0000005,6.1,2114,6.0,Blacksmithing,US,0
...,...,...,...,...,...,...
tt9916200,8.3,158,1.0,The Great Experiment,US,0
tt9916204,8.1,183,1.0,Better Angels,US,0
tt9916348,9.3,13,1.0,Ancient World Exposed,US,0
tt9916720,6.1,56,10.0,The Demonic Nun,US,0


Now we have an analyzable dataset, but ordering isn't telling us anything so let's remove it

In [68]:
df.drop(columns=['ordering'], inplace= True)

KeyError: "['ordering'] not found in axis"

In [69]:
df

,averageRating,numVotes,title,region,isOriginalTitle
tt0000001,5.6,1639,Carmencita,US,0
tt0000002,6.1,198,The Clown and His Dogs,US,0
tt0000005,6.1,2114,Blacksmithing Scene,US,0
tt0000005,6.1,2114,Blacksmith Scene #1,US,0
tt0000005,6.1,2114,Blacksmithing,US,0
...,...,...,...,...,...
tt9916200,8.3,158,The Great Experiment,US,0
tt9916204,8.1,183,Better Angels,US,0
tt9916348,9.3,13,Ancient World Exposed,US,0
tt9916720,6.1,56,The Demonic Nun,US,0


In [70]:
basics = pd.read_csv('Basics.tsv', sep='\t')
basics.head()

C:\Users\zbtin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [71]:
basics.shape

(7041952, 9)

In [73]:
df = df.join(basics.set_index('tconst'), how='left')
df

,averageRating,numVotes,title,region,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000001,5.6,1639,Carmencita,US,0,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
tt0000002,6.1,198,The Clown and His Dogs,US,0,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
tt0000005,6.1,2114,Blacksmithing Scene,US,0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000005,6.1,2114,Blacksmith Scene #1,US,0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
tt0000005,6.1,2114,Blacksmithing,US,0,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9916200,8.3,158,The Great Experiment,US,0,tvEpisode,The Great Experiment,The Great Experiment,0,2019,\N,43,Drama
tt9916204,8.1,183,Better Angels,US,0,tvEpisode,Better Angels,Better Angels,0,2019,\N,42,Drama
tt9916348,9.3,13,Ancient World Exposed,US,0,video,Ancient World Exposed,Ancient World Exposed,0,2019,\N,67,History
tt9916720,6.1,56,The Demonic Nun,US,0,short,The Nun 2,The Nun 2,0,2019,\N,10,"Comedy,Horror,Mystery"


In [75]:
df.titleType.value_counts()

movie           121613
tvEpisode        98318
short            59443
video            41774
tvSeries         30642
tvMovie          18723
tvSpecial         7411
videoGame         7315
tvMiniSeries      3312
tvShort           1862
Name: titleType, dtype: int64

We will only be looking at movies

In [76]:
movies = df['titleType']=='movie'
df = df[movies]
df

,averageRating,numVotes,title,region,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
tt0000009,5.9,154,Miss Jerry,US,0,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
tt0000147,5.2,355,The Corbett-Fitzsimmons Fight,US,0,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,20,"Documentary,News,Sport"
tt0000630,3.2,12,Hamlet,US,0,movie,Hamlet,Amleto,0,1908,\N,\N,Drama
tt0000679,5.2,37,The Fairylogue and Radio-Plays,US,0,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,\N,120,"Adventure,Fantasy"
tt0000886,5.1,21,"Hamlet, Prince of Denmark",US,0,movie,"Hamlet, Prince of Denmark",Hamlet,0,1910,\N,\N,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,6.8,471,Manoharam,US,0,movie,Manoharam,Manoharam,0,2019,\N,122,"Comedy,Drama"
tt9909228,5.8,20,Ximei,US,0,movie,Ximei,Ximei,0,2019,\N,98,Documentary
tt9913056,7.0,21,Swarm Season,US,0,movie,Swarm Season,Swarm Season,0,2019,\N,86,Documentary
tt9913660,5.3,35,No Apology,US,0,movie,No Apology,No Apology,0,2019,\N,102,Drama


In [79]:
df.drop(columns=['endYear'], inplace = True)
df

C:\Users\zbtin\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,averageRating,numVotes,title,region,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
tt0000009,5.9,154,Miss Jerry,US,0,movie,Miss Jerry,Miss Jerry,0,1894,45,Romance
tt0000147,5.2,355,The Corbett-Fitzsimmons Fight,US,0,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,20,"Documentary,News,Sport"
tt0000630,3.2,12,Hamlet,US,0,movie,Hamlet,Amleto,0,1908,\N,Drama
tt0000679,5.2,37,The Fairylogue and Radio-Plays,US,0,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,120,"Adventure,Fantasy"
tt0000886,5.1,21,"Hamlet, Prince of Denmark",US,0,movie,"Hamlet, Prince of Denmark",Hamlet,0,1910,\N,Drama
...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,6.8,471,Manoharam,US,0,movie,Manoharam,Manoharam,0,2019,122,"Comedy,Drama"
tt9909228,5.8,20,Ximei,US,0,movie,Ximei,Ximei,0,2019,98,Documentary
tt9913056,7.0,21,Swarm Season,US,0,movie,Swarm Season,Swarm Season,0,2019,86,Documentary
tt9913660,5.3,35,No Apology,US,0,movie,No Apology,No Apology,0,2019,102,Drama


In [80]:
crew = pd.read_csv('Crew.tsv', sep='\t')
crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [81]:
crew.shape

(7039263, 3)

In [82]:
df = df.join(crew.set_index('tconst'), how='left')
df

,averageRating,numVotes,title,region,isOriginalTitle,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,directors,writers
tt0000009,5.9,154,Miss Jerry,US,0,movie,Miss Jerry,Miss Jerry,0,1894,45,Romance,nm0085156,nm0085156
tt0000147,5.2,355,The Corbett-Fitzsimmons Fight,US,0,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,20,"Documentary,News,Sport",nm0714557,\N
tt0000630,3.2,12,Hamlet,US,0,movie,Hamlet,Amleto,0,1908,\N,Drama,nm0143333,nm0000636
tt0000679,5.2,37,The Fairylogue and Radio-Plays,US,0,movie,The Fairylogue and Radio-Plays,The Fairylogue and Radio-Plays,0,1908,120,"Adventure,Fantasy","nm0091767,nm0877783","nm0000875,nm0877783"
tt0000886,5.1,21,"Hamlet, Prince of Denmark",US,0,movie,"Hamlet, Prince of Denmark",Hamlet,0,1910,\N,Drama,nm0099901,nm0000636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tt9906644,6.8,471,Manoharam,US,0,movie,Manoharam,Manoharam,0,2019,122,"Comedy,Drama",nm7011994,\N
tt9909228,5.8,20,Ximei,US,0,movie,Ximei,Ximei,0,2019,98,Documentary,"nm0743417,nm0169214","nm0169214,nm0743417"
tt9913056,7.0,21,Swarm Season,US,0,movie,Swarm Season,Swarm Season,0,2019,86,Documentary,nm1502645,\N
tt9913660,5.3,35,No Apology,US,0,movie,No Apology,No Apology,0,2019,102,Drama,nm1067844,nm3014403


In [84]:
df.writers.value_counts()

\N                               11072
nm0765430                          102
nm0393250                           60
nm0000636                           58
nm0000095                           56
                                 ...  
nm1080741,nm0310018,nm0472406        1
nm0467942,nm0223804                  1
nm5098918,nm5209991                  1
nm2606006                            1
nm0441601,nm0635805                  1
Name: writers, Length: 73692, dtype: int64